# <font color='blue'>Comprehension of test code</font>

In [ ]:
import math

# Define default circuit parameters
default_circuit_parameters = {'param1': 0.01, 'param2': 1000}

# Define the dummy answer vector
dummy_answer_vector = [0, 1, 2, 2, 3, 4, 2, 1, 4, 4, 5, 4, 4, 1, 2, 6]

def evaluate_circuit(vector_size, circuit_vector, parameters=default_circuit_parameters):
    """
    This function takes a circuit vector and returns a performance value.
    The current version of the function is a dummy function that returns
    a performance value based on how close the circuit vector is to a predetermined answer vector.
    """
    performance = 0.0
    for i in range(vector_size):
        # dummy_answer_vector is a predetermined answer vector (same size as circuit_vector)
        performance += (20 - abs(circuit_vector[i] - dummy_answer_vector[i])) * 100.0
    return performance

# Overload function (delete if not needed)
# def evaluate_circuit(vector_size, circuit_vector):
#     return evaluate_circuit(vector_size, circuit_vector, default_circuit_parameters)

# Example usage
vector_size = 16
# circuit_vector = [0, 1, 2, 2, 3, 4, 2, 1, 4, 4, 5, 4, 4, 1, 2, 6]
circuit_vector = [0, 1, 2, 3, 4, 5, 6, 0, 1, 1, 2, 2, 3, 0, 1, 0]
performance = evaluate_circuit(vector_size, circuit_vector)
print(f"Performance: {performance}")


Performance: 29200.0


In [44]:
import numpy as np

def calculate_unit_flows(F_G, F_w):
    # 常量定义
    k_G_C = 0.004  # Gerardium in high grade concentrate stream
    k_w_C = 0.0002  # Waste in high grade concentrate stream
    k_G_I = 0.001  # Gerardium in intermediate grade concentrate stream
    k_w_I = 0.0003  # Waste in intermediate grade concentrate stream

    rho = 3000  # kg/m³, solid density
    phi = 0.1  # Volume fraction solids
    V = 10  # m³, cell volume

    if F_G + F_w == 0:
        return 0, 0, 0, 0, 0, 0

    # 停留时间
    tau = (phi * V) / ((F_G + F_w) / rho)

    # 回收率
    R_G_C = (k_G_C * tau) / (1 + k_G_C * tau + k_G_I * tau)
    R_w_C = (k_w_C * tau) / (1 + k_w_C * tau + k_w_I * tau)
    R_G_I = (k_G_I * tau) / (1 + k_G_C * tau + k_G_I * tau)
    R_w_I = (k_w_I * tau) / (1 + k_w_C * tau + k_w_I * tau)

    # 计算流量
    C_G = F_G * R_G_C
    C_w = F_w * R_w_C
    I_G = F_G * R_G_I
    I_w = F_w * R_w_I
    T_G = F_G * (1 - R_G_C - R_G_I)
    T_w = F_w * (1 - R_w_C - R_w_I)

    return C_G, C_w, I_G, I_w, T_G, T_w

def evaluate_circuit(circuit_vector, tolerance=1.0e-6, max_iterations=10000):
    num_units = (len(circuit_vector) - 1) // 3
    feed_unit = circuit_vector[0]

    # 初始化单元流量
    F_G = np.zeros(num_units)  # Gerardium feed rate
    F_w = np.zeros(num_units)  # Waste feed rate
    # F_G[feed_unit] = 10  # Initial gerardium feed rate
    # F_w[feed_unit] = 90  # Initial waste feed rate
    F_G = np.full(num_units, 10)  # Gerardium feed rate
    F_w = np.full(num_units, 90)  # Waste feed rate

    for iteration in range(max_iterations):
        # 初始化最终浓缩单元和尾矿单元的流量
        final_concentrate_G = 0
        final_concentrate_w = 0
        final_tails_G = 0
        final_tails_w = 0

        old_F_G = F_G.copy()
        old_F_w = F_w.copy()
        new_F_G = np.zeros(num_units)
        new_F_w = np.zeros(num_units)
        new_F_G[feed_unit] = 10  # Initial gerardium feed rate
        new_F_w[feed_unit] = 90  # Initial waste feed rate


        for i in range(num_units):
            C_G, C_w, I_G, I_w, T_G, T_w = calculate_unit_flows(F_G[i], F_w[i])

            conc_dest = circuit_vector[3 * i + 1]
            inter_dest = circuit_vector[3 * i + 2]
            tails_dest = circuit_vector[3 * i + 3]

            if conc_dest == num_units:
                final_concentrate_G += C_G
                final_concentrate_w += C_w
            elif conc_dest < num_units and conc_dest != feed_unit:
                new_F_G[conc_dest] += C_G
                new_F_w[conc_dest] += C_w

            if inter_dest == num_units:
                final_concentrate_G += I_G
                final_concentrate_w += I_w
            elif inter_dest < num_units and conc_dest != feed_unit:
                new_F_G[inter_dest] += I_G
                new_F_w[inter_dest] += I_w

            if tails_dest == num_units + 1:
                final_tails_G += T_G
                final_tails_w += T_w
            elif tails_dest < num_units and conc_dest != feed_unit:
                new_F_G[tails_dest] += T_G
                new_F_w[tails_dest] += T_w

        F_G, F_w = new_F_G, new_F_w

        if np.allclose(old_F_G, F_G, rtol=tolerance) and np.allclose(old_F_w, F_w, rtol=tolerance):
            break



    performance = 100 * final_concentrate_G - 750 * final_concentrate_w
    return performance, F_G, F_w, final_concentrate_G, final_concentrate_w

# 示例电路向量
# circuit_vector = [0, 1, 2, 2, 3, 0, 0, 1, 0, 4, num_units, num_units, num_units+1]
circuit_vector =[0, 1, 3, 2, 4, 4, 3, 1, 3, 6, 1, 1, 0, 5, 1, 1]
circuit_vector =[0, 1, 3, 3, 2, 2, 0, 4, 1, 1, 1, 0, 5]

performance, F_G, F_w, f_g, f_w = evaluate_circuit(circuit_vector)
print("Circuit Performance:", performance)

Circuit Performance: 110.24946574174113


In [42]:
def evaluate_circuit(circuit_vector, tolerance=1.0e-6, max_iterations=10000):
    num_units = (len(circuit_vector) - 1) // 3
    feed_unit = circuit_vector[0]

    # 初始化单元流量
    F_G = np.zeros(num_units)  # Gerardium feed rate
    F_w = np.zeros(num_units)  # Waste feed rate
    # F_G[feed_unit] = 10  # Initial gerardium feed rate
    # F_w[feed_unit] = 90  # Initial waste feed rate
    F_G = np.full(num_units, 10)  # Gerardium feed rate
    F_w = np.full(num_units, 90)  # Waste feed rate

    for iteration in range(max_iterations):
        # 初始化最终浓缩单元和尾矿单元的流量
        final_concentrate_G = 0
        final_concentrate_w = 0
        final_tails_G = 0
        final_tails_w = 0

        old_F_G = F_G.copy()
        old_F_w = F_w.copy()
        new_F_G = np.zeros(num_units)
        new_F_w = np.zeros(num_units)
        new_F_G[feed_unit] = 10  # Initial gerardium feed rate
        new_F_w[feed_unit] = 90  # Initial waste feed rate


        for i in range(num_units):
            C_G, C_w, I_G, I_w, T_G, T_w = calculate_unit_flows(F_G[i], F_w[i])

            conc_dest = circuit_vector[3 * i + 1]
            inter_dest = circuit_vector[3 * i + 2]
            tails_dest = circuit_vector[3 * i + 3]

            if conc_dest == num_units:
                final_concentrate_G += C_G
                final_concentrate_w += C_w
            elif conc_dest < num_units and conc_dest != feed_unit:
                new_F_G[conc_dest] += C_G
                new_F_w[conc_dest] += C_w

            if inter_dest == num_units:
                final_concentrate_G += I_G
                final_concentrate_w += I_w
            elif inter_dest < num_units and conc_dest != feed_unit:
                new_F_G[inter_dest] += I_G
                new_F_w[inter_dest] += I_w

            if tails_dest == num_units + 1:
                final_tails_G += T_G
                final_tails_w += T_w
            elif tails_dest < num_units and conc_dest != feed_unit:
                new_F_G[tails_dest] += T_G
                new_F_w[tails_dest] += T_w

        F_G, F_w = new_F_G, new_F_w

        if np.allclose(old_F_G, F_G, rtol=tolerance) and np.allclose(old_F_w, F_w, rtol=tolerance):
            break



    performance = 100 * final_concentrate_G - 750 * final_concentrate_w
    return performance, F_G, F_w, final_concentrate_G, final_concentrate_w

# 示例电路向量
# circuit_vector = [0, 1, 2, 2, 3, 0, 0, 1, 0, 4, num_units, num_units, num_units+1]
circuit_vector =[0, 1, 3, 2, 4, 4, 3, 1, 3, 6, 1, 1, 0, 5, 1, 1]
circuit_vector =[0, 1, 3, 3, 2, 2, 0, 4, 1, 1, 1, 0, 5]

performance, F_G, F_w, f_g, f_w = evaluate_circuit(circuit_vector)
print("Circuit Performance:", performance)

Circuit Performance: 110.24946574174113


In [43]:
f_g

1.503315088033363